In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime

from summer.utils import ref_times_to_dti

from autumn.tools.plots.utils import REF_DATE
from autumn.tools.runs import ManagedRun
from autumn.tools.runs.calibration.utils import get_posteriors
from autumn.tools.utils.pandas import pdfilt
from autumn.settings.region import Region

## Specify the runs of interest

In [ ]:
run_ids, mrs, pbis, results, targets = {}, {}, {}, {}, {}
run_ids["west-metro"] = "covid_19/west-metro/1635166421/153a938"
run_ids["barwon-south-west"] = "covid_19/barwon-south-west/1635203341/e17f4a0"
run_ids["gippsland"] = "covid_19/gippsland/1635166443/153a938"
run_ids["grampians"] = "covid_19/grampians/1635166473/153a938"
run_ids["hume"] = "covid_19/hume/1635166677/153a938"
run_ids["loddon-mallee"] = "covid_19/loddon-mallee/1635166635/153a938"
run_ids["north-east-metro"] = "covid_19/north-east-metro/1635204422/94ea65b"
run_ids["south-east-metro"] = "covid_19/south-east-metro/1635166421/153a938"

In [ ]:
scenario_names = ["baseline", "roadmap scenario"]
plot_left_date = datetime.date(2021, 8, 2)

In [ ]:
for region in Region.VICTORIA_SUBREGIONS:
    mrs[region] = ManagedRun(run_ids[region])
    pbis[region] = mrs[region].powerbi.get_db()
    results[region] = pbis[region].get_uncertainty()
    targets[region] = pbis[region].get_targets()

In [ ]:
full_runs = {}
for region in Region.VICTORIA_SUBREGIONS:
    full_runs[region] = mrs[region].full_run.get_derived_outputs()

In [ ]:
plot_left_date = datetime.date(2021, 9, 1)
fig, axes = plt.subplots(4, 2, figsize=(12, 20), sharex="all")
output = "notifications"
for i_ax, axis in enumerate(axes.reshape(-1)):
    region = Region.VICTORIA_SUBREGIONS[i_ax]
    full_run = full_runs[region]
    scenario_chain = (full_run["scenario"] == 0) & (full_run["chain"] == 0)
    for i_run in full_run[scenario_chain]["run"].unique():
        selection = full_run[(full_run["run"] == i_run) & scenario_chain]
        axis.plot(ref_times_to_dti(REF_DATE, selection["times"]), selection[output])
    if output in targets[region]:
        axis.scatter(targets[region].index, targets[region][output], color="k", s=10, alpha=0.5, zorder=10)
    axis.set_title(region)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date)

In [ ]:
def plot_outputs(output, scenario):
    fig, axes = plt.subplots(4, 2, figsize=(12, 20), sharex="all")
    colour = (0.2, 0.2, 0.8)
    for i_ax, axis in enumerate(axes.reshape(-1)):
        region = Region.VICTORIA_SUBREGIONS[i_ax]
        results_df = results[region][(output, scenario)]
        indices = results_df.index
        axis.fill_between(indices, results_df[0.01], results_df[0.99], color=colour, alpha=0.1)
        axis.fill_between(indices, results_df[0.025], results_df[0.975], color=colour, alpha=0.5)
        axis.fill_between(indices, results_df[0.25], results_df[0.75], color=colour, alpha=0.7)
        axis.plot(indices, results_df[0.500], color="b", linestyle="--")
        if output in targets[region]:
            axis.scatter(targets[region].index, targets[region][output], color="k", alpha=0.5, s=10)
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=plot_left_date)
        axis.set_title(region)
    fig.suptitle(f"{output} under {scenario_names[scenario]} conditions")
    fig.tight_layout()

In [ ]:
outputs = (
    "notifications", "infection_deaths", "hospital_admissions", "hospital_occupancy", "icu_admissions", "icu_occupancy"
)

for scenario in [0, 1]:
    for output in outputs:
        plot_outputs(output, scenario)
        
# plot_outputs("notifications", 0)

In [ ]:
mcmc_params = mrs[region].calibration.get_mcmc_params()

fig, axes = plt.subplots(3, 2, figsize=(12, 18))
for i_ax, axis in enumerate(axes.reshape(-1)[:5]):
    axis.hist(mcmc_params.iloc[:, i_ax])
    axis.set_title(mcmc_params.columns[i_ax])